# **Đặt vấn đề:**
- Trong bối cảnh đô thị hóa nhanh chóng và nhu cầu sở hữu nhà ở ngày càng tăng, việc dự báo giá nhà trở thành một trong những vấn đề quan trọng và cấp bách nhất trong lĩnh vực bất động sản, ảnh hưởng sâu rộng đến nền kinh tế và đời sống của mọi người. Việc dự báo giá nhà đóng vai trò then chốt trong quản lý và đầu tư bất động sản, giúp các nhà phân tích, nhà đầu tư và người mua nhà có thể ra quyết định hiệu quả. 
- Song song với đó, sự phát triển nhanh chóng của khoa học dữ liệu và trí tuệ nhân tạo, việc áp dụng các kỹ thuật này vào lĩnh vực bất động sản đang trở thành một xu hướng tất yếu. Bất động sản, với bản chất phức tạp và sự biến động mạnh mẽ của thị trường, đòi hỏi các phương pháp phân tích và dự báo tiên tiến để hỗ trợ các quyết định kinh doanh chính xác và kịp thời. Đây là một thử thách lớn đối với việc phân tích dữ liệu, yêu cầu phải kết hợp nhiều kỹ thuật và mô hình học máy để xử lý và dự báo dữ liệu. Dự án "Dự báo Giá Nhà" chính là một bài tập ứng dụng tổng hợp các kỹ năng và kiến thức đã học trong môn Nhập môn Khoa học Dữ liệu, nhằm giải quyết một vấn đề thực tế và có ý nghĩa kinh tế - xã hội sâu sắc.


# **Create URL Tails, City Dictionary and District Dictionary**

#### **1. Crawl đuôi URL và tạo dictionary cho Tỉnh/Thành phố**

In [1]:
from bs4 import BeautifulSoup
import urllib.request

def download_html(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    return html

In [2]:
first_url = 'https://batdongsan.vn/ban-nha/'
html = download_html(first_url)
soup = BeautifulSoup(html, 'lxml')

In [3]:
# Tìm phần tử <select> bên trong <div class="city">
select_tag = soup.find('div', {'class': 'city'}).find('select')
# Tìm tất cả các thẻ <option> bên trong nó.
option_tags = select_tag.find_all('option')
# Lấy danh sách các giá trị (value) từ các thẻ <option> (bỏ qua option đầu tiên có giá trị rỗng)
city_dictionary={}
city_tails=[]
for option in option_tags:
    value = option.get('value')
    text = option.text.strip()
    if value and text:  # Chỉ thêm vào từ điển nếu cả value và text đều tồn tại
        city_dictionary[value] = text
        city_tails.append(value)

#### **2. Crawl đuôi URL và tạo dictionary cho Quận/Huyện**

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Khởi tạo trình duyệt 
options = webdriver.ChromeOptions()
options.add_argument('--headless') 
driver = webdriver.Chrome(options=options)

# Dictionary để lưu kết quả
district_tails = {}
district_dictionary = {}

# Lặp qua mỗi tỉnh thành trong cityDataFrame
for city_tail in city_tails:
    # Tạo URL cho từng tỉnh thành
    second_url = first_url[:-1] + "-" + city_tail
    driver.get(second_url)
    
    # Chờ cho danh sách quận/huyện được tải đầy đủ
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "select[name='district'] option:not([value=''])"))
    )

    # Phân tích HTML
    soup = BeautifulSoup(driver.page_source, 'lxml')
    select_tag = soup.find('div', {'class': 'district'}).find('select')
    option_tags = select_tag.find_all('option')
    district_list=[]
    for option in option_tags:
        value = option.get('value')
        text = option.text.strip()
        if value and text:  # Chỉ thêm vào từ điển nếu cả value và text đều tồn tại
            district_dictionary[value] = text
            district_list.append(value)

    # Lưu danh sách quận/huyện vào dictionary
    district_tails[city_tail] = district_list

# Đóng trình duyệt
driver.close()

#### **3. Kiểm tra lại kết quả**

In [5]:
print(city_tails)

['ha-noi', 'ho-chi-minh', 'an-giang', 'ba-ria-vung-tau', 'bac-giang', 'bac-kan', 'bac-lieu', 'bac-ninh', 'ben-tre', 'binh-duong', 'binh-dinh', 'binh-phuoc', 'binh-thuan', 'ca-mau', 'can-tho', 'cao-bang', 'da-nang', 'dak-lak', 'dak-nong', 'dien-bien', 'dong-nai', 'dong-thap', 'gia-lai', 'ha-giang', 'ha-nam', 'ha-tinh', 'hai-duong', 'hai-phong', 'hau-giang', 'hoa-binh', 'hung-yen', 'khanh-hoa', 'kien-giang', 'kon-tum', 'lai-chau', 'lam-dong', 'lang-son', 'lao-cai', 'long-an', 'nam-dinh', 'nghe-an', 'ninh-binh', 'ninh-thuan', 'phu-tho', 'phu-yen', 'quang-binh', 'quang-nam', 'quang-ngai', 'quang-ninh', 'quang-tri', 'soc-trang', 'son-la', 'tay-ninh', 'thai-binh', 'thai-nguyen', 'thanh-hoa', 'thua-thien-hue', 'tien-giang', 'tra-vinh', 'tuyen-quang', 'vinh-long', 'vinh-phuc', 'yen-bai']


In [6]:
print(city_dictionary)

{'ha-noi': 'Hà Nội', 'ho-chi-minh': 'TP Hồ Chí Minh', 'an-giang': 'An Giang', 'ba-ria-vung-tau': 'Bà Rịa Vũng Tàu', 'bac-giang': 'Bắc Giang', 'bac-kan': 'Bắc Kạn', 'bac-lieu': 'Bạc Liêu', 'bac-ninh': 'Bắc Ninh', 'ben-tre': 'Bến Tre', 'binh-duong': 'Bình Dương', 'binh-dinh': 'Bình Định', 'binh-phuoc': 'Bình Phước', 'binh-thuan': 'Bình Thuận', 'ca-mau': 'Cà Mau', 'can-tho': 'Cần Thơ', 'cao-bang': 'Cao Bằng', 'da-nang': 'Đà Nẵng', 'dak-lak': 'Đắk Lắk', 'dak-nong': 'Đắk Nông', 'dien-bien': 'Điện Biên', 'dong-nai': 'Đồng Nai', 'dong-thap': 'Đồng Tháp', 'gia-lai': 'Gia Lai', 'ha-giang': 'Hà Giang', 'ha-nam': 'Hà Nam', 'ha-tinh': 'Hà Tĩnh', 'hai-duong': 'Hải Dương', 'hai-phong': 'Hải Phòng', 'hau-giang': 'Hậu Giang', 'hoa-binh': 'Hòa Bình', 'hung-yen': 'Hưng Yên', 'khanh-hoa': 'Khánh Hòa', 'kien-giang': 'Kiên Giang', 'kon-tum': 'Kon Tum', 'lai-chau': 'Lai Châu', 'lam-dong': 'Lâm Đồng', 'lang-son': 'Lạng Sơn', 'lao-cai': 'Lào Cai', 'long-an': 'Long An', 'nam-dinh': 'Nam Định', 'nghe-an': 'Nghệ

In [7]:
print(district_tails)

{'ha-noi': ['ba-dinh-ha-noi', 'ba-vi-ha-noi', 'bac-tu-liem-ha-noi', 'cau-giay-ha-noi', 'chuong-my-ha-noi', 'dan-phuong-ha-noi', 'dong-anh-ha-noi', 'dong-da-ha-noi', 'gia-lam-ha-noi', 'ha-dong-ha-noi', 'hai-ba-trung-ha-noi', 'hoai-duc-ha-noi', 'hoan-kiem-ha-noi', 'hoang-mai-ha-noi', 'long-bien-ha-noi', 'me-linh-ha-noi', 'my-duc-ha-noi', 'nam-tu-liem-ha-noi', 'phu-xuyen-ha-noi', 'phuc-tho-ha-noi', 'quoc-oai-ha-noi', 'soc-son-ha-noi', 'son-tay-ha-noi', 'tay-ho-ha-noi', 'thach-that-ha-noi', 'thanh-oai-ha-noi', 'thanh-tri-ha-noi', 'thanh-xuan-ha-noi', 'thuong-tin-ha-noi', 'ung-hoa-ha-noi'], 'ho-chi-minh': ['binh-chanh-ho-chi-minh', 'binh-tan-ho-chi-minh', 'binh-thanh-ho-chi-minh', 'can-gio-ho-chi-minh', 'cu-chi-ho-chi-minh', 'go-vap-ho-chi-minh', 'hoc-mon-ho-chi-minh', 'nha-be-ho-chi-minh', 'phu-nhuan-ho-chi-minh', 'quan-1-ho-chi-minh', 'quan-10-ho-chi-minh', 'quan-11-ho-chi-minh', 'quan-12-ho-chi-minh', 'quan-2-ho-chi-minh', 'quan-3-ho-chi-minh', 'quan-4-ho-chi-minh', 'quan-5-ho-chi-minh',

In [8]:
print(district_dictionary)

{'ba-dinh-ha-noi': 'Ba Đình', 'ba-vi-ha-noi': 'Ba Vì', 'bac-tu-liem-ha-noi': 'Bắc Từ Liêm', 'cau-giay-ha-noi': 'Cầu Giấy', 'chuong-my-ha-noi': 'Chương Mỹ', 'dan-phuong-ha-noi': 'Đan Phượng', 'dong-anh-ha-noi': 'Đông Anh', 'dong-da-ha-noi': 'Đống Đa', 'gia-lam-ha-noi': 'Gia Lâm', 'ha-dong-ha-noi': 'Hà Đông', 'hai-ba-trung-ha-noi': 'Hai Bà Trưng', 'hoai-duc-ha-noi': 'Hoài Đức', 'hoan-kiem-ha-noi': 'Hoàn Kiếm', 'hoang-mai-ha-noi': 'Hoàng Mai', 'long-bien-ha-noi': 'Long Biên', 'me-linh-ha-noi': 'Mê Linh', 'my-duc-ha-noi': 'Mỹ Đức', 'nam-tu-liem-ha-noi': 'Nam Từ Liêm', 'phu-xuyen-ha-noi': 'Phú Xuyên', 'phuc-tho-ha-noi': 'Phúc Thọ', 'quoc-oai-ha-noi': 'Quốc Oai', 'soc-son-ha-noi': 'Sóc Sơn', 'son-tay-ha-noi': 'Sơn Tây', 'tay-ho-ha-noi': 'Tây Hồ', 'thach-that-ha-noi': 'Thạch Thất', 'thanh-oai-ha-noi': 'Thanh Oai', 'thanh-tri-ha-noi': 'Thanh Trì', 'thanh-xuan-ha-noi': 'Thanh Xuân', 'thuong-tin-ha-noi': 'Thường Tín', 'ung-hoa-ha-noi': 'Ứng Hòa', 'binh-chanh-ho-chi-minh': 'Bình Chánh', 'binh-tan